In [2]:
%reload_ext autoreload
%autoreload 2
import datetime
import pathlib
import numpy
import os
import sklearn.preprocessing
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from classifier import classification_data

In [3]:
cfg = {
    "fold": 5,
    "n_labels": 5,
    "path_base": "../features",
    "path_out": "out",
    "test_size": 0.2,
    "train_size": 0.8,
    "random_state": 1
}
file_input = os.path.join(cfg["path_base"], "handcraft", "MATLAB", "GRAYSCALE", "256", "surf64.txt")

In [4]:
surf = numpy.loadtxt(file_input)
dataset = "surf"
n_samples, n_features = surf.shape
n_features = n_features - 1
x, y = surf[0:, 0:n_features], surf[:, n_features]
x_normalized = sklearn.preprocessing.StandardScaler().fit_transform(x)

print(x.shape, y.shape)

(375, 257) (375,)


In [ ]:
kf = sklearn.model_selection.KFold(n_splits=cfg["fold"], shuffle=True, random_state=cfg["random_state"])
current_datetime = datetime.datetime.now().strftime("%d-%m-%Y-%H-%M-%S")
path = os.path.join(cfg["path_out"], current_datetime, dataset)
pathlib.Path(path).mkdir(parents=True, exist_ok=True)

for pca in (128, 256, -1):
    x = x_normalized
    if pca > 0:
        x = sklearn.decomposition.PCA(n_components=pca).fit_transform(x)

    if pca == -1:
        pca = n_features

    classification_data(cfg, dataset, file_input, list(kf.split(x)), pca, n_samples, path, x, y)